In [1]:
pip install opencv-python

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install face-recognition

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install keyboard

Note: you may need to restart the kernel to use updated packages.


In [4]:
# IMPORT THE NECESSARY FILES

import cv2
import numpy as np
import matplotlib.pyplot as plt
import face_recognition
import os
import keyboard
from datetime import datetime
%matplotlib inline

In [5]:
# SET UP THE IMAGES' DATABASE AND THEIR RESPECTIVE LABELS' DATABASE

path = 'PDB'
pictureDB = os.listdir(path)
imageDB = []
namesDB = []
#print(pictureDB)
for i in pictureDB:
    #print(f'{path}/'+i)
    currImg = cv2.imread(f'{path}/'+i)
    imageDB.append(currImg)
    namesDB.append(os.path.splitext(i)[0])

In [6]:
# EXTRACT THE FACE ENCODINGS FOR ALL THE FACES IN THE IMAGE DATABASE

def encodings(imageDB):
    encodedDB = []
    for image in imageDB:
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        imageEncoding = face_recognition.face_encodings(image)[0]
        encodedDB.append(imageEncoding)
    return encodedDB

knowEncodings = encodings(imageDB)
print('Encoding Complete')

Encoding Complete


In [7]:
def setup():
    with open('Attendance.csv','w+') as f:
        f.writelines('Names,Time')
    f.close()
    
def attendance(name):
    #setup()
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        #if 'Name,Time' not in f:
        #    f.writelines('Name,Time')
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            dtString = now.strftime('%H:%M:%S')
            f.writelines(f'\n{name},{dtString}')
    f.close()

In [8]:
# VIDEO

setup()

webCam = cv2.VideoCapture(0)

while True:
    succesfulFrameRead,frame=webCam.read()
    
    if not succesfulFrameRead:
        print("webcam not found")
        break
    #frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    #frameG=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    frameS = cv2.resize(frame,(0,0),None,0.25,0.25)
    facesCurrFrame = face_recognition.face_locations(frameS)
    encodesCurrFrame = face_recognition.face_encodings(frameS,facesCurrFrame)
    
    for encodeFace,faceLoc in zip(encodesCurrFrame,facesCurrFrame):
        matches = face_recognition.compare_faces(knowEncodings,encodeFace)
        faceDis = face_recognition.face_distance(knowEncodings,encodeFace)
        #print(faceDis)
        #print(matches)
        matchIndex = np.argmin(faceDis)   
    
        if matches[matchIndex]:
            name = namesDB[matchIndex]
            #print(name)
            y1,x2,y2,x1 = faceLoc
            y1,x2,y2,x1 = y1*4,x2*4,y2*4,x1*4
            cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(frame,(x1,y2+35),(x2,y2),(255,255,255),cv2.FILLED)
            cv2.putText(frame,name,(x1+15,y2+25),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),2)
            attendance(name)
    
    cv2.imshow('Face Detector',frame)
    cv2.waitKey(1)
    if keyboard.is_pressed('q'):
        print("Quitting application")
        break
        
webCam.release()
cv2.destroyAllWindows()

Quitting application
